# Evaluate rough Bergomi model part 1-6
After running all notebooks containing the Bergomi models (train_rbergomi_1...6-complete.ipynb and train_rbergomi_1...6.ipynb) with or without the training parts, you can run this notebook to get some evaluations on these models.

In [1]:
import os
import plaidml.keras
plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import keras
keras.backend.backend()

'plaidml'

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import os, sys, pandas as pd, numpy as np
wd = os.getcwd()

path_seperator = os.path.sep

sys.path.append(wd + path_seperator + 'optimizers')
from AdamW import AdamW
from QHAdam import QHAdam

# Evaluation of all models

In [3]:
percentiles = [0.5, 0.75, 0.95, 0.99, 0.999, 0.9999, 1.0]
optimizers=[keras.optimizers.Adam(), AdamW(), QHAdam()]
all_optimizers = ["Adam reference"] + optimizers

dfs = []
for batch_size in [32, 128]:
    df = pd.DataFrame(columns = percentiles)
    for optimizer in all_optimizers:
        if type(optimizer) == str:
            file_end = ''
        else: 
            file_end = '_' + optimizer.__class__.__name__ + '_' + str(batch_size) 

        predictions_list= []
        for model_part in [1,2,3,4,5,6]:
            predictions_list.append(pd.read_csv(wd + path_seperator + 'data' + path_seperator + 'neural_network_weights' + path_seperator + 'rbergomi' + path_seperator + 'predictions' + path_seperator + 'pred' + str(model_part) + file_end + '.csv', delimiter=",", header = None, skiprows=1))

        predictions = pd.concat(predictions_list)
        difference = abs(predictions.iloc[:,0]-predictions.iloc[:,1])
        precent = difference / predictions.iloc[:,0]

        q_list = []
        for percentile in [0.5, 0.75, 0.95, 0.99, 0.999, 0.9999, 1.0]:
            q_list.append(np.quantile(precent, percentile))

        df = df.append(pd.DataFrame([np.array(q_list) * 100], columns=percentiles))
        
    df = df.reset_index(drop = True)
    df = df.rename(index={0: all_optimizers[0], 1: all_optimizers[1].__class__.__name__, 2: all_optimizers[2].__class__.__name__, 3: all_optimizers[3].__class__.__name__ })
    df.columns.name = "Optimizer / percentile"
    dfs.append(df)

INFO:plaidml:Opening device "metal_amd_radeon_pro_450.0"


## Absolute relative errors with batch size of 32 during the first training part

In [4]:
dfs[0].style.set_table_attributes('style="font-size: 15px"')

Optimizer / percentile,0.5,0.75,0.95,0.99,0.999,0.9999,1.0
Adam reference,0.178474,0.369853,1.028009,2.438705,7.551112,18.095318,388.898734
Adam,0.173790,0.356891,0.992514,2.401761,7.664393,19.242425,192.109987
AdamW,0.162849,0.340128,0.968896,2.308853,7.160003,18.563294,76.540189
QHAdam,0.298262,0.548361,1.438738,3.023577,7.640942,17.186868,236.413384


## Absolute relative errors with batch size of 128 during the first training part

In [5]:
dfs[1].style.set_table_attributes('style="font-size: 15px"')

Optimizer / percentile,0.5,0.75,0.95,0.99,0.999,0.9999,1.0
Adam reference,0.178474,0.369853,1.028009,2.438705,7.551112,18.095318,388.898734
Adam,0.219357,0.445389,1.251674,2.816530,7.720693,18.839204,93.069545
AdamW,0.198392,0.424861,1.244589,2.890164,7.756074,18.896796,65.272836
QHAdam,0.359358,0.796853,2.492686,5.306227,12.649679,23.488396,68.019882


# Mean training times per epoches 

In [6]:
times = []
for batch_size in [32, 128]:
    for optimizer in optimizers:
        file_end = '_' + optimizer.__class__.__name__ + '_' + str(batch_size) 

        predictions_list= []
        for model_part in [1,2,3,4,5,6]:
            predictions_list.append(pd.read_csv(wd + path_seperator + 'data' + path_seperator + 'neural_network_weights' + path_seperator + 'rbergomi' + path_seperator 
                                         + 'epoch_times' + path_seperator + 'rbergomi_model_' + str(model_part) + file_end + '_mean_epoch_time.csv', delimiter=",", header = None, skiprows=1))
    
        times.append(pd.concat(predictions_list))

## Training time evaluation of both parts of the training with the adam optimizer with batch size of 32 for the first training part

In [7]:
df = times[0].reset_index(drop=True)
df.columns = ['Training part 1', 'Training modelpart 2']
df.columns.name = "Model part"
df.index = np.arange(1, len(df)+1)
df

Model part,Training part 1,Training modelpart 2
1,38.780025,1.703516
2,39.698801,1.628800
3,33.197755,2.529615
4,37.989159,3.027042
5,31.428143,1.634130
6,35.727938,3.728810


In [8]:
print(np.mean(df))

Model part
Training part 1         36.136970
Training modelpart 2     2.375319
dtype: float64


## Training time evaluation of both parts of the training with the adam optimizer with batch size of 128 for the first training part

In [9]:
df = times[4].reset_index(drop=True)
df.columns = ['Training part 1', 'Training part 2']
df.columns.name = "Model part"
df.index = np.arange(1, len(df)+1)
df

Model part,Training part 1,Training part 2
1,12.178476,1.731428
2,12.811610,1.674868
3,14.396309,2.892544
4,14.539418,3.156233
5,12.827549,1.903525
6,17.317564,4.972611


In [10]:
print(np.mean(df))

Model part
Training part 1    14.011821
Training part 2     2.721868
dtype: float64


## Training time evaluation of  all models

In [11]:
time_df = []
for t,op,ba in zip(times, optimizers + optimizers, [32,32,32,128,128,128]):
    time_df.append([op.__class__.__name__, ba] + np.mean(t).tolist())
pd.DataFrame(time_df, columns=["optimizer","first batch size","model part 1","model part 2"])

,optimizer,first batch size,model part 1,model part 2
0,Adam,32,36.136970,2.375319
1,AdamW,32,36.836511,2.498834
2,QHAdam,32,38.543118,2.484594
3,Adam,128,13.493508,2.611904
4,AdamW,128,14.011821,2.721868
5,QHAdam,128,14.203553,2.540415
